In [ ]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
from django.core.exceptions import ObjectDoesNotExist, FieldError
file = "archiv\data\Database_Vorarlberg_Correction_Progress.xlsx"
df = pd.read_excel(file, "ENTITY")[3:].fillna('False')

In [ ]:
failed_sites = []
for i, row in df.iterrows():
    temp_item = None
    temp_site = None
    if row['*SiteID'] != 'False':
        try:
            temp_site = Site.objects.get(identifier=row['*SiteID'])
        except ObjectDoesNotExist:
            failed_sites.append(row['*SiteID'])
    if temp_site:
        if row['*Entity ID'] != 'False':
            temp_item, _ = ArchEnt.objects.get_or_create(identifier=row['*Entity ID'])
            temp_item.site_id = temp_site
            
            if row['Entity Name'] != 'False':
                temp_item.name = row['Entity Name']
            
            # needs to be applied on a cumulativ column
#             if row['Entity type'] != 'False':
#                 x = None
#                 try:
#                     x = SkosConcept.objects.get(pref_label=row['Entity type'])
#                 except ObjectDoesNotExist:
#                     x = None
#                 if x:
#                     temp_item.ent_type = x

            
            if row['Topography'] != 'False':
                x = None
                for x in row['Topography'].split(','):
                    x = x.strip()
                    try:
                        y = SkosConcept.objects.get(pref_label=x)
                    except ObjectDoesNotExist:
                        y = None
                    if y:
                        temp_item.topography.add(y)
            
            if row['Entity Type Certainty'] != 'False':
                temp_item.type_certainty = row['Entity Type Certainty']

            if row['Dating certainty'] != 'False':
                temp_item.dating_certainty = row['Dating certainty']
            
            if row['Location certainty'] != 'False':
                temp_item.location_certainty = row['Location certainty']

            if row['Comment'] != 'False':
                temp_item.comment = row['Comment']
            
            temp_item.save()
    

In [ ]:
# ArchEnt.objects.all().delete()